In [1]:
import cv2
import numpy as np
import pyautogui
import time
import streamlink

In [2]:
#  Вычисление окружности
def is_inside(point, center, radius):
    """ This function returns True if point is inside/on the boundary, False otherwise """
    eqn = (point[0] - center[0]) ** 2 + (point[1] - center[1])**2 - radius**2
    return eqn <= 0

In [3]:
def calc_dist(pt1, pt2):
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    dst = ((x2 - x1)**2 + (y2-y1)**2)**0.5
    return dst


In [16]:
img1 = cv2.imread("my_screenshot1.png", cv2.CV_8UC1)
circles = cv2.HoughCircles(img1, cv2.HOUGH_GRADIENT, 1, 100,
                          param1=100, param2=70,
                          minRadius=25, maxRadius=100)

res = np.zeros(img1.shape)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        #cv2.circle(res, center, 1, (0, 100, 100), 3)
        # circle outline
        radius = i[2] # - 7
        cv2.circle(res, center, radius, (200, 200, 255), 1)

cv2.imshow("res", res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
img_path = "my_screenshot.png"
temp_path = "template.png"
img  = cv2.imread(img_path, cv2.CV_8UC1)
temp = cv2.imread(temp_path, cv2.CV_8UC1)
res = cv2.matchTemplate(img, temp, cv2.TM_CCOEFF_NORMED)
treshold = 0.99
loc = np.where(res >= treshold)
y = loc[0][0]
x = loc[1][0]
w, h = np.shape(temp)
roi_rect = img[y:y+h, x:x+w]

circles = cv2.HoughCircles(roi_rect, cv2.HOUGH_GRADIENT, 1, h / 8,
                          param1=70, param2=25,
                          minRadius=25, maxRadius=35)

res = np.zeros(roi_rect.shape)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        #cv2.circle(res, center, 1, (0, 100, 100), 3)
        # circle outline
        radius = i[2] - 7
        cv2.circle(roi_rect, center, radius, (255, 0, 255), 1)

In [77]:
filterd_image  = cv2.medianBlur(roi_rect,3)
ret, thresh = cv2.threshold(filterd_image, 75, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img_contours = np.zeros(filterd_image.shape)
contours_sorted = sorted(contours, key=len, reverse=True)
max_dist = 0
max_pt = (0, 0)
for pt in contours_sorted[1]:
    if(calc_dist(pt[0], center) > max_dist):
        max_pt = pt[0]
        max_dist = calc_dist(pt[0], center)
cv2.circle(img_contours, center, radius, (150, 150, 150), 1)
cv2.circle(img_contours, center, 2, (200, 200, 200), -1);
cv2.circle(img_contours, max_pt, 2, (200, 200, 200), -1);

In [76]:
max_pt

array([19, 33], dtype=int32)

In [79]:
cv2.namedWindow("roi", cv2.WINDOW_NORMAL)
cv2.imshow("roi", img_contours)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
for y in range(h):
    for x in range(w):
        if is_inside((x, y),center, radius):
            continue
        else:
            roi_rect[y][x] = 0
    

In [140]:
print("w=", w, "h=", h, "size=", np.shape(roi_rect))

w= 83 h= 78 size= (78, 83)


In [26]:
filterd_image  = cv2.medianBlur(roi_rect,3)
ret, thresh = cv2.threshold(filterd_image, 75, 255, cv2.THRESH_BINARY)




In [58]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img_contours = np.zeros(filterd_image.shape)

# draw the contours on the empty image


In [59]:
contours_sorted = sorted(contours, key=len, reverse=True)

In [60]:
cv2.drawContours(img_contours, contours_sorted[1], -1, (255,255,255), 1)

cv2.imshow('contours', img_contours) # выводим итоговое изображение в окно

cv2.waitKey()
cv2.destroyAllWindows()

In [170]:
cv2.imshow('thresh', thresh) # выводим итоговое изображение в окно

cv2.waitKey()
cv2.destroyAllWindows()

In [152]:
cv2.imshow("res", roi_rect)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
cv2.imshow("res", res)
cv2.waitKey(0)
cv2.destroyAllWindows()